In [29]:
import pygame
import random

In [30]:
# Initialize Pygame
pygame.init()

# Window setup
width, height = 600, 400
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Palette Snake")

# Game settings
block_size = 20
speed = 5
clock = pygame.time.Clock()

# Food shape options
SHAPES = ['circle', 'diamond', 'square']

In [31]:
# Generate random color
def random_color():
    bright_colors = [
        (255, 69, 58),    # bright red
        (255, 149, 0),    # orange
        (255, 214, 10),   # yellow
        (52, 199, 89),    # bright green
        (0, 122, 255),    # sky blue
        (255, 45, 85),    # hot pink
        (175, 82, 222),   # purple
        (90, 200, 250)    # cyan
    ]
    return random.choice(bright_colors)

# Draw food on screen
def draw_food(shape, color, x, y):
    if shape == 'circle':
        pygame.draw.circle(win, color, (x + block_size // 2, y + block_size // 2), block_size // 2)

    elif shape == 'square':
        pygame.draw.rect(win, color, (x, y, block_size, block_size))

    elif shape == 'diamond':
        points = [
            (x + block_size // 2, y),                        # top
            (x + block_size, y + block_size // 2),           # right
            (x + block_size // 2, y + block_size),           # bottom
            (x, y + block_size // 2)                         # left
        ]
        pygame.draw.polygon(win, color, points)

# Draw the snake, including snake head and snake body
# Draw the snake head, the head always remains the same
def draw_snake_head(x, y):
    # Colors
    head_color = (100, 149, 237)      # Blue
    eye_white = (255, 255, 255)
    pupil_black = (30, 30, 30)

    # Head shape: rounded rectangle using two half-circles + center rectangle
    radius = block_size // 2
    center_x = x + radius
    center_y = y + radius

    # Horizontal capsule-style head
    head_width = block_size + 10
    head_height = block_size
    head_rect = pygame.Rect(center_x - head_width // 2, center_y - head_height // 2, head_width, head_height)
    pygame.draw.ellipse(win, head_color, head_rect)

    # Eye (centered in the upper half)
    eye_radius = 7
    eye_center = (center_x, center_y - 4)
    pygame.draw.circle(win, eye_white, eye_center, eye_radius)

    # Pupil
    pupil_radius = 3
    pygame.draw.circle(win, pupil_black, eye_center, pupil_radius)

# Draw the snake body
def draw_snake(snake_parts):
    # Draw the custom head
    draw_snake_head(snake_parts[0][0], snake_parts[0][1])

    # Draw the rest of the body, the body adds the food shape and color every time it eats one food
    for part in snake_parts[1:]:
        draw_food(part[2], part[3], part[0], part[1])

# If game over, show the 'Game over' text
def show_game_over_text():
    font = pygame.font.SysFont("Arial", 36, bold=True)
    text = font.render("Game Over", True, (0, 0, 0))  # black text
    text_rect = text.get_rect(center=(width // 2, height // 2))
    win.blit(text, text_rect)
    pygame.display.update()

# generate new food in random color, shape and position
def gen_new_food():
    food_x = random.randint(0, (width - block_size) // block_size) * block_size
    food_y = random.randint(0, (height - block_size) // block_size) * block_size
    food_shape = random.choice(SHAPES)
    food_color = random_color()
    return food_x, food_y, food_shape, food_color

In [32]:
# Main game loop
def game_loop():
    x, y = width // 2, height // 2 # x and y represent the position of the snake head
    dx, dy = 0, 0  # No movement until arrow key is pressed

    # Snake body which contain all food shapes, each food part = [x, y, shape, color]
    snake_parts = [[x, y, 'square', (0, 255, 0)]]

    # Generate new food 
    [food_x, food_y, food_shape, food_color]=gen_new_food()

    running = True
    while running:
        clock.tick(speed)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP and dy == 0:
                    # if dy !=0, it means the snake is already going up or down, then the key does not work
                    dx, dy = 0, -block_size
                elif event.key == pygame.K_DOWN and dy == 0:
                    dx, dy = 0, block_size
                elif event.key == pygame.K_LEFT and dx == 0:
                    dx, dy = -block_size, 0
                elif event.key == pygame.K_RIGHT and dx == 0:
                    dx, dy = block_size, 0

        if dx == 0 and dy == 0: # if no key is pressed
            # Waiting for first move
            win.fill((200, 255, 200)) # make the background color light green
            draw_food(food_shape, food_color, food_x, food_y)
            draw_snake(snake_parts)
            pygame.display.update()
            continue  # quit this iteration and enter the next iteration to wait for key pressing

        # if an valid key is pressed, update snake head position
        x += dx
        y += dy

        # Wall collision
        if x < 0 or x >= width or y < 0 or y >= height:
            show_game_over_text()
            pygame.time.delay(3000)  # pause for 3 seconds
            pygame.quit()
            return

        # Self collision
        for part in snake_parts[1:]:
            if x == part[0] and y == part[1]:
                show_game_over_text()
                pygame.time.delay(3000)  # pause for 3 seconds
                pygame.quit()
                return

        # Check if food is eaten
        ate_food = (x == food_x and y == food_y)

        if ate_food:
            # Move the head position to the food position
            snake_parts[0][0:2] = [x, y]
            
            # Add food with its shape and color
            add_food = [x-dx, y-dy, food_shape, food_color]
            # Insert the food, stick it behind the head
            snake_parts.insert(1, add_food)
        else: 
            # if not ate food, move all snake parts one step forward          
            for i in range(len(snake_parts)-1,0,-1):
                # original position(x,y):   h 1 2 3 4 
                # modified position(x,y):   1 2 3 4 h
                snake_parts[i][0:2]=snake_parts[i-1][0:2] 
                # modified position(x,y): h 1 2 3 4    
            snake_parts[0][0:2]=[x,y]

        if ate_food:
            # Generate new food
            [food_x, food_y, food_shape, food_color]=gen_new_food()

        # Draw everything
        win.fill((200, 255, 200)) # make the background color light green
        draw_food(food_shape, food_color, food_x, food_y)
        draw_snake(snake_parts)
        pygame.display.update()

    pygame.quit()

# Run the game
game_loop()
